#
  - 3. Syndigo Mapping MachineLearning

In [1]:
def unique_no_nan(x):
    return x.dropna().unique()

In [2]:
def constrcut_dic():
    key = unique_no_nan(syndigo_mapped['Level 1'])
    value = list( map(lambda x : re.sub('&|/', '', x), key))
    value = list( map(lambda x : re.sub(' +', '_', x), value))
    value = list(map(str.lower, value))
    #value = ['-'.join(['tfidf'+ str]), value]

    print(f'key: {key}') 
    print(f'value: {value}')
    return dict(zip(key, value)) 

In [3]:
def prepare_data_1():
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    #print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    print(f"syndigo_mapped['Level 1'].value_counts() {syndigo_mapped['Level 1'].value_counts(dropna = False)}")    
    return syndigo_mapped, pim_gtin_mapped

In [4]:
def prepare_data_2(level__1):
    level_1 = level__1
    subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]
    #print(f'1. type(subset_df)= {type(subset_df)}, len(subset_df)= {len(subset_df)} subset_df.shape {subset_df.shape}  \n subset_df.head {subset_df.head()}') 
    series = syndigo_mapped[syndigo_mapped['Level 1']==level__1]['Level 2'].value_counts(ascending=True)
    #print(f' series {series} \n type(series)= {type(series)}, len(series)= {len(series)} \n series.tolist() {series.tolist()} ')      
    if series.tolist():
        subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series==1].index.tolist())].index)
    #print(f'2. type(subset_df)= {type(subset_df)}, len(subset_df)= {len(subset_df)} subset_df.shape {subset_df.shape}  \n subset_df.head {subset_df.head()}')
    
    vect_subset = TfidfVectorizer(ngram_range = (1,2), max_features = 50000)
    X_subset = vect_subset.fit_transform(subset_df.ITEM_SUBCOM_text) #scipy.sparse._csr.csr_matrix
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    return  X_subset, y_subset, subset_df, level2_id_map

In [25]:
def proc_predict(level__1):
     global A_train, A_test, B_train, B_test, X_train, X_test, y_train, y_test
     A_train, A_test, B_train, B_test = train_test_split(subset_df.GTIN.tolist(), subset_df['Level 2'].tolist(), test_size=0.1, stratify=y_subset, random_state=42)
     X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size= 0.1, stratify=y_subset, random_state=42)
     #lr_tf = LogisticRegressionCV(cv = 5, multi_class = 'multinomial', solver = 'saga', n_jobs=-1, max_iter=10000000)
     lr_tf = LogisticRegression(C = 100, multi_class = 'multinomial', solver = 'saga', n_jobs=-1)
     
     print(f"Training - \"{level__1}\"")
     lr_tf.fit(X_train, y_train)
     preds = lr_tf.predict(X_test)
     #print(f"type(preds) {type(preds)} len(preds) {len(preds)} \n preds[0:10] {preds[0:10]}")
     preds_lrtf = preds
     probs = lr_tf.predict_proba(X_test)
     #print(f"type(probs) {type(probs)} len(probs) {len(probs)} \n probs[0:10] {probs[0:10]}")   
     preds_train = lr_tf.predict(X_train)
     #print(f"type(preds_train) {type(preds_train)} len(preds_train) {len(preds_train)} \n preds_train[0:10] {preds_train[0:10]} ")   
     probs_train = lr_tf.predict_proba(X_train)
     #print(f"type(probs_train) {type(probs_train)} len(probs_train) {len(probs_train)} \n probs_train[0:10] {probs_train[0:10]}")   
     #print(classification_report(y_test, preds_test,labels = lr_tf.classes_, target_names = [id2_level_map[i] for i in lr_tf.classes_]))
     print(f"Done Training - \"{level__1}\"")
     return preds, preds_lrtf, probs, preds_train, probs_train  

In [6]:
def proc_append():
    
    test_new_proobs = []
    
    #print(f'len(preds)  {len(preds)}')
    for i in range(len(preds)):
        test_new_proobs.append(probs[i][np.argsort(probs[i])][::-1][:1].tolist())
    test_new_proobs = [element for sublist in list(test_new_proobs) for element in sublist]
    #print(f'test_new_proobs {test_new_proobs}')
    
    train_new_proobs = []
    
    #print(f'len(preds_train)  {len(preds_train)}')
    for i in range(len(preds_train)):
         test_new_proobs.append(probs_train[i][np.argsort(probs_train[i])][::-1][:1].tolist())
    train_new_proobs = [element for sublist in list(train_new_proobs) for element in sublist]
    #print(f'train_new_proobs {train_new_proobs}')  
    
    # process testLevels & trainLevelss 
    testLevels = []
    for j in y_test:
        testLevels.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    #print(f'testLevels {testLevels [0:10]}')
    testLevelss = []
    for j in preds:
        testLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    #print(f'testLevelss {testLevelss [0:10]}')
    
    trainLevelss = []
    for j in preds_train:
        trainLevelss.append([i for i in level2_id_map if level2_id_map[i]==j][0])
    
    return  test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss

In [7]:
def proc_save(level__1, filenamee):
    print("---------------------------------------------\FINAL COUNTS:\n---------------------------------------------")
    print("1)", len(A_test + A_train))
    print("2)", len(['Test']*len(A_test) + ['Train']*len(A_train)))
    print("3)", len(B_test + B_train))
    print("4)", len(testLevels + B_train))
    print("5)", len(testLevelss + trainLevelss))
    print("6)", int(len(test_new_proobs + train_new_proobs)))
    
    data = {
        'GTIN' : A_test + A_train,
        'Source': ['Test']*len(A_test) + ['Train']*len(A_train),
        'Actual Level 2' : B_test + B_train,
        'Actuals' : testLevels + B_train,
        'Predictions' : testLevelss + trainLevelss,
        'Scores' : test_new_proobs + train_new_proobs,
        'L1 Name' : level__1
    }
    
    df = pd.DataFrame(data)
    df = df.merge(pim_gtin_mapped[['GTIN', 'ITM_ID', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD',
        'REC_DPT_DSC', 'DPT_CD', 'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD',
        'SUBCOM_DSC', 'VND_ECOM_DSC']], on='GTIN', how='left')
    df = df[['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
        'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
        'VND_ECOM_DSC', 'Source', 'Actual Level 2', 'Actuals', 'Predictions', 'Scores', 'L1 Name']]
    df.to_csv(DBFO + filenamee + '.csv', index=False)
    print(f"{filenamee}.csv   --file created successfully")
    return

In [8]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
import re

In [9]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [11]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()

syndigo_mapped['Level 1'].value_counts() Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
NaN                                           1132
Not classified                                1018
Hardware                                       923
Arts & Crafts                            

In [12]:
dic_level1= constrcut_dic()


key: ['Food / Beverages' 'Health & Beauty' 'Beer / Wine / Spirits'
 'Kitchen & Bathroom' 'Cleaning & Janitorial' 'Home & Venue Decoration'
 'Livestock & Pet Supplies' 'Toys / Games / Hobbies' 'Childcare'
 'Tobacco Products' 'Apparel' 'Gardening & Outdoors'
 'Sports & Outdoor Recreation Equipment' 'Office Supplies'
 'Plumbing & Water Service' 'Lighting & Fans' 'Appliances' 'Electronics'
 'Not classified' 'Automotive' 'Tools' 'Hardware' 'Books & Videos'
 'Arts & Crafts' 'Electrical' 'Heating / Ventilation / Air Conditioning'
 'Hospitality Supplies' 'Furniture' 'Power Sports'
 'Power Transmission & Motion Control' 'Marine' 'Flooring'
 'Paints & Coatings' 'Building Supplies' 'Musical Instruments'
 'Material Handling' 'Agricultural Equipment']
value: ['food_beverages', 'health_beauty', 'beer_wine_spirits', 'kitchen_bathroom', 'cleaning_janitorial', 'home_venue_decoration', 'livestock_pet_supplies', 'toys_games_hobbies', 'childcare', 'tobacco_products', 'apparel', 'gardening_outdoors', 'spor

In [26]:
#dic_test={"Hospitality Supplies":"hospitality_spplies", "Tobacco Products":"bacco_products"}
#dic_test={"Food / Beverages":"food_beverages"}

for level__1, filenamee in dic_level1.items():
    X_subset, y_subset, subset_df, level2_id_map =  prepare_data_2(level__1)
    preds, preds_lrtf, probs, preds_train, probs_train = proc_predict(level__1)
    test_new_proobs, train_new_proobs, testLevels, testLevelss, trainLevelss = proc_append()
    proc_save(level__1, filenamee) 

Training - "Food / Beverages"


C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Done Training - "Food / Beverages"
---------------------------------------------\FINAL COUNTS:
---------------------------------------------
1) 131593
2) 131593
3) 131593
4) 131593
5) 131593
6) 131593
food_beverages.csv   --file created successfully


## wip

In [29]:
# type(A_train) # list
A_train[0:10]

[810047600510.0,
 44300107304.0,
 11110851277.0,
 719212799243.0,
 819597011685.0,
 27000380406.0,
 819215020754.0,
 11150026000.0,
 72220900491.0,
 43000077412.0]

In [31]:
#type(A_test) # list
A_test[0:10]

[22045063666.0,
 72830006019.0,
 20735420041.0,
 795404134047.0,
 70074630601.0,
 41224705562.0,
 78000113464.0,
 41757025434.0,
 767778001027.0,
 620133001707.0]

In [33]:
#type(B_train) # list
B_train[0:10]

['Beverages',
 'Grocery',
 'Grocery',
 'Grocery',
 'Grocery',
 'Grocery',
 'Beverages',
 'Grocery',
 'Grocery',
 'Bakery / Deli']